# Datenanalyse Seattle Library

In [1]:
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import csv

## Daten runterladen

In [62]:
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2022%20AND%20isbn%20is%20not%20null&$limit=100"
local_file = "checkout.csv"
urllib.request.urlretrieve(url, local_file)

('checkout.csv', <http.client.HTTPMessage at 0x7df5e3841a90>)

In [63]:
df_seattle = pd.read_csv(local_file)

In [64]:
df_seattle

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
0,Physical,Horizon,BOOK,2022,9,6,Great day for up! Pictures by Quentin Blake.,"0394829131, 0394929136","Seuss, Dr.",Stories in rhyme,"Beginner Books,",[1974]
1,Digital,OverDrive,AUDIOBOOK,2022,9,1,The Scientist and the Spy: A True Story of Chi...,9780593166819,Mara Hvistendahl,"Business, Politics, True Crime, Nonfiction",Books on Tape,2020
2,Digital,OverDrive,EBOOK,2022,9,3,The Flame and the Flower,9780061743696,Kathleen E. Woodiwiss,"Fiction, Romance, Historical Fiction",HarperCollins Publishers Inc.,2006
3,Digital,OverDrive,EBOOK,2022,9,8,Fifty Words for Rain: A Novel,9781524746377,Asha Lemmie,"Fiction, Literature, Historical Fiction","Penguin Group (USA), Inc.",2020
4,Physical,Horizon,BOOK,2022,9,5,How to code a rollercoaster / by Josh Funk ; i...,"0425292037, 9780425292037","Funk, Josh","Amusement parks Juvenile fiction, Computer pro...","Viking,",[2019]
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Digital,OverDrive,EBOOK,2022,9,5,Bean by Bean: A Cookbook: More than 175 Recipe...,9780761168706,Crescent Dragonwagon,"Cooking & Food, Reference, Nonfiction","Hachette Digital, Inc.",2012
96,Digital,OverDrive,AUDIOBOOK,2022,9,1,3 Dozen Poems: From the Writer's Almanac (unab...,9781598879452,Garrison Keillor,"Fiction, Poetry",HighBridge Company,2010
97,Digital,OverDrive,EBOOK,2022,9,1,Roar: A Dinosaur Tour,9781524767006,Michael Paul,"History, Juvenile Nonfiction, Nature, Science","Random House, Inc.",2018
98,Digital,OverDrive,EBOOK,2022,9,7,River of Fire: Warriors: A Vision of Shadows S...,9780062386564,Erin Hunter,"Fantasy, Juvenile Fiction, Juvenile Literature",HarperCollins Publishers Inc.,2018


## ISBN